In [8]:
import pandas as pd
import requests, json

# API
city_name = 'Melbourne'
API_KEY = '6456aee2ec2d47df861f5e544387a73c'
lat = -37.8142454
lon = 144.9631732

complete_url = f"https://pro.openweathermap.org/data/2.5/forecast/hourly?lat={lat}&lon={lon}&appid={API_KEY}"

response = requests.get(complete_url)

x = response.json()

In [9]:
df = pd.DataFrame(x['list'])
df.head(2)

,dt,main,weather,clouds,wind,visibility,pop,sys,dt_txt,rain
0,1733320800,"{'temp': 288.7, 'feels_like': 288.4, 'temp_min...","[{'id': 800, 'main': 'Clear', 'description': '...",{'all': 0},"{'speed': 0.7, 'deg': 270, 'gust': 2.81}",10000,0.0,{'pod': 'n'},2024-12-04 14:00:00,NaN
1,1733324400,"{'temp': 288.73, 'feels_like': 288.38, 'temp_m...","[{'id': 800, 'main': 'Clear', 'description': '...",{'all': 0},"{'speed': 1.18, 'deg': 332, 'gust': 2.3}",10000,0.0,{'pod': 'n'},2024-12-04 15:00:00,NaN


In [12]:
from datetime import datetime
import pytz
import pandas as pd


df = pd.DataFrame(x['list'])
df

# Extract relevant columns
df['temp'] = df['main'].apply(lambda x: x['temp'])
df['humidity'] = df['main'].apply(lambda x: x['humidity'])
# replace NAn with 0 in df['rain']
df['rain'] = df['rain'].fillna(0)
df['clouds'] = df['clouds'].apply(lambda x: x['all'])

# Convert Unix timestamp to Melbourne local time
melbourne_tz = pytz.timezone("Australia/Melbourne")
df['datetime_melbourne'] = df['dt'].apply(
    lambda x: pytz.utc.localize(datetime.utcfromtimestamp(x)).astimezone(melbourne_tz)
)

# Set datetime as index
df.set_index('datetime_melbourne', inplace=True)

# Keep only relevant columns
df = df[['temp', 'humidity', 'rain', 'clouds']]

# Display the table
df.head(2)

,temp,humidity,rain,clouds
datetime_melbourne,,,,
2024-12-05 01:00:00+11:00,288.70,80,0,0
2024-12-05 02:00:00+11:00,288.73,78,0,0


In [14]:
import holidays

# Create a list of public holidays in Victoria
australia_holidays = holidays.Australia(state='VIC')

# Add a new column to indicate whether each date is a public holiday
df['IsPublicHoliday'] = [
    1 if date in australia_holidays else 0 for date in df.index.date
]

df['Hour'] = df.index.hour
df['Weekday'] = df.index.weekday + 1
df['Month'] = df.index.month
df['Season'] = df['Month'].map({12: 'Winter', 1: 'Winter', 2: 'Winter',
                                                      3: 'Spring', 4: 'Spring', 5: 'Spring',
                                                      6: 'Summer', 7: 'Summer', 8: 'Summer',
                                                      9: 'Fall', 10: 'Fall', 11: 'Fall'})

# Display the updated DataFrame
df.head(3)

,temp,humidity,rain,clouds,IsPublicHoliday,Hour,Weekday,Month,Season
datetime_melbourne,,,,,,,,,
2024-12-05 01:00:00+11:00,288.70,80,0,0,0,1,4,12,Winter
2024-12-05 02:00:00+11:00,288.73,78,0,0,0,2,4,12,Winter
2024-12-05 03:00:00+11:00,288.62,76,0,0,0,3,4,12,Winter


# When Creating dummies they need to match the reference groups of the models and the coloumns too

1733328000